<a href="https://colab.research.google.com/github/looktei/NPPROJECT1/blob/main/KRAS_autodock_gpu_G13D__BI_2493_8ONV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **1. Install packages and dependencies**

# %%capture
import contextlib
import timeit
import torch

# -- Start install --
start = timeit.default_timer()

with open("/content/installation.log", "w") as i:
    with contextlib.redirect_stdout(i):
        !wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
        !chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
        !bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        !conda install -c conda-forge mamba -y

        #pip
        !pip install py3Dmol==2.0.0.post2
        !pip install pybel==0.15.5
        !pip install rdkit-pypi==2022.9.5
        !pip install pdb2pqr==3.5.2
        import os
        os.chdir('/content')
        !git clone https://github.com/tieulongphan8995/dock_util.git


        # Vina
        !wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64 -O vina
        !chmod u+x vina

        # qvina2
        %cd /content
        !git clone https://github.com/QVina/qvina.git
        %cd /content/qvina/bin
        !chmod u+x qvina2.1
        %cd /content

        #smina
        !wget https://downloads.sourceforge.net/project/smina/smina.static
        !chmod u+x smina.static
        !./smina.static --help



        # install conda
        os.chdir('/content')
        !mamba install -c conda-forge -c bioconda mgltools=1.5.7 biopython=1.78 \
          openbabel=3.1.0 plip=2.2.2 zlib=1.2.11 xlsxwriter=3.0.3 -y

        !rm -r /content/sample_data /content/condacolab_install.log

stop = timeit.default_timer()
# -- End install --

time_elp = stop - start

print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")

+ Operation completed
+ Time elapsed: 03m 57s


In [ ]:
#@title **2. Import Python modules**

import os
import sys
sys.path.append('/content/dock_util')
from grid_box import *
from result_process import *


import ast
import math
import plip
import timeit
import shutil
import py3Dmol
import contextlib
import xlsxwriter
import urllib.request

import numpy as np
import pandas as pd

from google.colab import drive, files
from tqdm.notebook import tqdm
from openbabel import pybel
from Bio.PDB import PDBIO, PDBParser
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, PandasTools
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex
#from AutoDockTools.Utilities24 import prepare_ligand4, prepare_receptor4
print(f"+ Imported done")
print(f"+ Environment ready for molecular docking")

No module Pymol
+ Imported done
+ Environment ready for molecular docking


In [ ]:
#@title **3. Create Working folder**
#@markdown Enter a **< Job Name >** without space.\
#@markdown This create a folder for protein, ligand, experimental and docking.

Job_name = "KRAS-G13D-BI_2493" #@param {type:"string"}
assert not Job_name == "", "Do not leave this blank."
assert not any(c == "/" or c == "." for c in Job_name), "Disallowed characters."

DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, Job_name)
PRT_FLD = os.path.join(WRK_DIR, "PROTEIN")
LIG_FLD = os.path.join(WRK_DIR, "LIGAND")
EXP_FLD = os.path.join(WRK_DIR, "EXPERIMENTAL")
DCK_FLD = os.path.join(WRK_DIR, "DOCKING")
INT_FLD = os.path.join(WRK_DIR, "INTERACTION")

folders = [WRK_DIR, PRT_FLD, LIG_FLD, EXP_FLD, DCK_FLD, INT_FLD]

for f in folders:
    if os.path.exists(f):
        print(f"+ {os.path.basename(f)} folder already exists")
    if not os.path.exists(f):
        os.mkdir(f)
        print(f"+ {os.path.basename(f)} folder created")

+ KRAS-G13D-BI_2493 folder created
+ PROTEIN folder created
+ LIGAND folder created
+ EXPERIMENTAL folder created
+ DOCKING folder created
+ INTERACTION folder created


In [ ]:
#@title **4. Set up utilities**
#@markdown This creates important variables and functions that will be utilized
#@markdown throughout the docking study.
import os

%alias vina /content/vina
%alias qvina2 /content/qvina/bin/qvina2.1
%alias smina ./smina.static


COLORS = ["red", "orange", "yellow", "lime", "green", "cyan", "teal", "blue",
          "violet", "purple", "pink", "gray", "brown", "white", "black"]

BOND_DICT = {"hydrophobic": ["0x59e382", "GREEN"],
             "hbond": ["0x59bee3", "LIGHT BLUE"],
             "waterbridge": ["0x4c4cff", "BLUE"],
             "saltbridge": ["0xefd033", "YELLOW"],
             "pistacking": ["0xb559e3", "PURPLE"],
             "pication": ["0xe359d8", "VIOLET"],
             "halogen": ["0x59bee3", "LIGHT BLUE"],
             "metal": ["0xe35959", "ORANGE"]}




import os
import os
import sys


import ast
import math
import plip
import timeit
import shutil
import py3Dmol
import contextlib
import xlsxwriter
import urllib.request

import numpy as np
import pandas as pd

from google.colab import drive, files
from tqdm.notebook import tqdm
from openbabel import pybel
from Bio.PDB import PDBIO, PDBParser
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, PandasTools
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex

COLORS = ["red", "orange", "yellow", "lime", "green", "cyan", "teal", "blue",
          "violet", "purple", "pink", "gray", "brown", "white", "black"]

BOND_DICT = {"hydrophobic": ["0x59e382", "GREEN"],
             "hbond": ["0x59bee3", "LIGHT BLUE"],
             "waterbridge": ["0x4c4cff", "BLUE"],
             "saltbridge": ["0xefd033", "YELLOW"],
             "pistacking": ["0xb559e3", "PURPLE"],
             "pication": ["0xe359d8", "VIOLET"],
             "halogen": ["0x59bee3", "LIGHT BLUE"],
             "metal": ["0xe35959", "ORANGE"]}

class Hide:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Biological assembly detector and extractor
def chain_profilier(inputFile):
    parser = PDBParser()
    structure = parser.get_structure("X", inputFile)
    chain_obj = [s for s in structure.get_chains()]
    chain_IDs = [c.get_id() for c in chain_obj]
    chain_len = len(chain_obj)
    return [chain_len, chain_IDs, chain_obj]

def seperate_chain(inputFile):
    io = PDBIO()
    name = os.path.basename(inputFile)[:-4]
    dir = os.path.dirname(inputFile)
    chain_info = chain_profilier(inputFile)
    chain_len, chain_IDs, chain_obj = chain_info[0], chain_info[1], chain_info[2]
    print(f"+ Chains detected: {chain_len} ({', '.join(chain_IDs)})")
    for ID,chain in zip(chain_IDs, chain_obj):
        chain_pdb = name + "_" + ID + ".pdb"
        chain_pdb_file = os.path.join(dir, chain_pdb)
        io.set_structure(chain)
        io.save(chain_pdb_file)
        print(f"+ {chain_pdb} ==> {os.path.basename(dir)} folder")

# Binding interactions profiler
def interaction_profiler(inputPL):
    protlig = PDBComplex()
    INTER = {}
    INTER_TYPE = []
    ID = os.path.basename(inputPL[:-7])
    CSV_Ifile = os.path.join(INT_FLD, f"{ID[:-2]}/{ID}_inter.csv")
    COL = ["RESNR", "RESTYPE", "RESCHAIN", "RESNR_LIG", "RESTYPE_LIG",
           "RESCHAIN_LIG", "DIST", "LIGCARBONIDX", "PROTCARBONIDX", "LIGCOO",
           "PROTCOO"]
    INTER_DF = pd.DataFrame(columns=COL)

    protlig.load_pdb(inputPL)
    LIG = protlig.ligands[0]
    protlig.characterize_complex(LIG)

    BD_keys = BOND_DICT.keys()
    BSI_key = f":".join([str(x) for x in LIG.members[0]])
    BSI_val = protlig.interaction_sets[BSI_key]
    BSR = BindingSiteReport(BSI_val)
    INTER[BSI_key] = {k: [getattr(BSR, f"{k}_features")] +
                      getattr(BSR, f"{k}_info")
                      for k in tuple(BD_keys)}
    for BD_key in list(BD_keys):
        DF = pd.DataFrame.from_records(
            INTER[BSI_key][BD_key][1:],
            columns=INTER[BSI_key][BD_key][0])
        if not DF.empty:
            INTER_TYPE.extend([BD_key.upper()]*len(DF))
            INTER_DF = INTER_DF.append(DF)
    INTER_DF = INTER_DF.assign(BOND=INTER_TYPE)
    INTER_DF.reset_index(drop=True, inplace=True)
    INTER_DF.to_csv(CSV_Ifile, index=False)

def view_interaction(inputCSV, result="summary"):
    DIST_CALC = []
    COLOR = []
    MIDCOO = []
    interaction = pd.read_csv(inputCSV, converters={
        "LIGCOO": lambda x: ast.literal_eval(str(x)),
        "PROTCOO": lambda x: ast.literal_eval(str(x)),
        "BOND": lambda x: x.lower()})
    for LC, PC, BT in zip(interaction["LIGCOO"], interaction["PROTCOO"],
                          interaction["BOND"]):
        COLOR.append(BOND_DICT[BT][1])
        p1 = np.array([LC[0], LC[1], LC[2]])
        p2 = np.array([PC[0], PC[1], PC[2]])
        squared_dist = np.sum((p1-p2)**2, axis=0)
        dist = np.round(np.sqrt(squared_dist) ,2)
        DIST_CALC.append(dist)
        mid_x = np.round((LC[0] + PC[0]) / 2, 2)
        mid_y = np.round((LC[1] + PC[1]) / 2, 2)
        mid_z = np.round((LC[2] + PC[2]) / 2, 2)
        p_mid = (mid_x, mid_y, mid_z)
        MIDCOO.append(p_mid)
    interaction["BOND"] = interaction["BOND"].str.upper()
    interaction["COLOR"] = COLOR
    interaction["MIDCOO"] = MIDCOO
    interaction["DIST_CALC"] = DIST_CALC
    interaction["RESNAME"] = interaction["RESTYPE"] + interaction["RESNR"].astype(str)

    summary = ["RESNR", "RESTYPE", "DIST_CALC", "BOND", "COLOR"]
    py3dmol = ["RESNR", "DIST_CALC", "LIGCOO", "PROTCOO", "MIDCOO", "BOND"]
    if result == "summary":
        df = interaction[summary]
    if result == "py3Dmol":
        df = interaction[py3dmol]
    return df

# Py3Dmol model viewer
def VIEW_PROT(inputP, model="Cartoon", color="white", focusRes="",
              focusResColor="white", addStick=False, addLine=False,
              showChain=False, showResLabel=False, showVDW=False,
              outline=False):
    # Variables
    mview = py3Dmol.view(1000,1500)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein Model
    count = 1
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    mol = open(inputP, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {model: {cType: color}})

    # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showChain and chainLen > 1:
        for n,c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carbon"}})
            mview.addLabel(
                f"Chain {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": res}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": prot_model})
    mview.show()

def VIEW_ELIG(inputE, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(1000, 1500)

    # Experimental ligand model
    count = 1
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    mol = open(inputE, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": elig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.zoomTo({"model": elig_model})
    mview.show()

def VIEW_LIG(inputL, showHs=False, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(1000, 1500)

    # Ligand model
    count = 1
    lig_model = count
    print(f"+ Showing {os.path.basename(inputL)}")
    smi = open(inputL, "r").read()
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)

    # Remove Hs
    if not showHs:
        mol = Chem.RemoveHs(mol)
    mblock = Chem.MolToMolBlock(mol)
    mview.addModel(mblock, "mol")
    mview.setStyle(
        {"model": lig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": lig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.show()

def VIEW_GRID(inputP, inputE, focusRes, center, size=[10, 10, 10]):
    # Variable
    mview = py3Dmol.view(1000, 1500)

    # Grid box
    bxi, byi, bzi = center[0], center[1], center[2]
    bxf, byf, bzf = size[0], size[1], size[2]
    print(f"+ Center: X {center[0]}  Y {center[1]}  Z {center[2]}")
    print(f"+ Size: W {size[0]}  H {size[1]}  D {size[2]}")
    mview.addBox(
        {"center":{"x":bxi, "y":byi, "z":bzi},
        "dimensions": {"w":bxf, "h":byf, "d":bzf},
        "color": "skyBlue", "opacity": 0.7})

    # Protein model
    count = 1
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {"cartoon": {"color": "white"}})

    # Experimental ligand model
    count += 1
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    molB = open(inputE, "r").read()
    mview.addModel(molB, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "greenCarbon"}})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": "orangeCarbon", "radius": 0.15}})
        mview.addResLabels(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"alignment": "bottomLeft",
            "showBackground": False,
            "inFront": True,
            "fixed": False,
            "fontSize": 14,
            "fontColor": "0x000000",
            "screenOffset": {"x": 15, "y": 15}})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": elig_model})
    mview.show()

def VIEW_PILE(inputP, inputPP, inputL, inputE, inputCSV, model="Cartoon",
              color="white", focusRes="", focusResColor="white",
              showInter=False, showSubunit=False, showResLabel=False,
              showVDW=False, showPartProt=False, showExpLig=False,
              showLig=False, showAllPose=False, showBestPose=False,
              outline=False):
    # Variables
    resUQ = []
    mview = py3Dmol.view(1000,1500)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein model
    count = 1
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model":prot_model},
        {model: {cType: color}})

  # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showSubunit and chainLen > 1:
        for n, c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carboon"}})
            mview.addLabel(
                f"Subunit {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    if focusRes != "":
        resUQ.extend([int(r) for r in focusRes.split(",")])
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": resUQ}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": resUQ}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Interactions
    if showInter:
        interaction = view_interaction(inputCSV, result="py3Dmol")
        RESNR = interaction["RESNR"]
        DIST_CALC = interaction["DIST_CALC"]
        LIGCOO = interaction["LIGCOO"]
        PROTCOO = interaction["PROTCOO"]
        MIDCOO = interaction["MIDCOO"]
        BOND = interaction["BOND"]
        for RN,DC,LC,PC,MC,BT in zip(RESNR, DIST_CALC, LIGCOO, PROTCOO, MIDCOO, BOND):
            BT = BT.lower()
            if RN not in resUQ:
                resUQ.append(RN)
                mview.addStyle(
                    {"and": [{"model": prot_model}, {"resi": RN}]},
                    {"stick": {"colorscheme": "whiteCarbon", "radius": 0.15}})
                mview.addResLabels(
                    {"and": [{"model": prot_model}, {"resi": RN}]},
                    {"alignment": "bottomLeft", "showBackground": False,
                     "inFront": True, "fixed": False, "fontSize": 14,
                     "fontColor": "0x000000", "screenOffset": {"x": 15, "y": 15}})
                mview.addCylinder(
                    {"start": {"x": LC[0], "y": LC[1], "z": LC[2]},
                     "end": {"x": PC[0], "y": PC[1], "z": PC[2]},
                     "radius": 0.05, "fromCap": 1, "toCap": 1,
                     "color": BOND_DICT[BT][0], "dashed": True})
                mview.addLabel(
                    str(DC) + " Å",
                    {"position": {"x": MC[0], "y": MC[1], "z": MC[2]},
                     "alignment": "bottomLeft", "inFront": False, "fixed": False,
                     "backgroundColor": BOND_DICT[BT][0], "fontSize": 10,
                     "screenOffset": {"x": 5, "y": 5}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Partner protein model
    if showPartProt and os.path.exists(inputPP):
        count += 1
        pprot_model = count
        print(f"+ Showing {os.path.basename(inputPP)} (yellow)")
        molB = open(inputPP, "r").read()
        mview.addModel(molB, "pdb")
        mview.setStyle(
            {"model": pprot_model},
            {"cartoon": {"color": "yellow"}})
        mview.addStyle(
            {"model": pprot_model},
            {"stick": {"colorscheme": "yellowCarbon"}})

    # Experimental ligand model
    if showExpLig:
        count += 1
        elig_model = count
        print(f"+ Showing {os.path.basename(inputE)} (gray)")
        molC = open(inputE, "r").read()
        mview.addModel(molC, "pdb")
        mview.setStyle(
            {"model": elig_model},
            {"stick": {"color": "gray"}})

    # Ligand model
    if showLig:
        count += 1
        lig_model = count
        print(f"+ Showing {os.path.basename(inputL)} (orange)")
        molD = open(inputL, "r").read()
        mview.addModel(molD, "pdb")
        mview.setStyle(
            {"model": lig_model},
            {"stick": {"colorscheme": "orangeCarbon"}})

    # Show poses of selected ligand
    if showAllPose:
        pose = sorted([ os.path.join(os.path.dirname(inputL), f) for f in os.listdir(os.path.dirname(inputL)) if f.endswith(".pdb") ])
        pose.remove(inputL[len(DCK_FLD)+1:]) if inputL[len(DCK_FLD)+1:] in pose else None
        for f in pose:
            count += 1
            pose_model = count
            fs = "".join(f)
            mol5 = open(os.path.join(DCK_FLD, fs), "r").read()
            mview.addModel(mol5, "pdb")
            mview.setStyle(
                {"model": pose_model},
                {"stick": {"color": "blue", "opacity": 0.5, "radius": 0.2}})

    if outline:
        mview.setViewStyle(
            {"style": "outline", "color": "black", "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.enableFog(True)
    mview.show()


# **5.Target protein structure preparation from RCSB Protein Data Bank (PDB)**

In [ ]:
#@title **5.1. Generate protein PDB file**
#@markdown Enter the **< PDB ID >** to download targeted protein.

PDB_ID = "8ONV" # @param {type:"string"}
PDB_pdb = PDB_ID + ".pdb"
PDB_pdb_Pfile = os.path.join(PRT_FLD, PDB_pdb)

urllib.request.urlretrieve(f"http://files.rcsb.org/download/{PDB_pdb}", PDB_pdb_Pfile)
print(f"+ PDB downloaded: {PDB_pdb} ==> PROTEIN folder")

+ PDB downloaded: 8ONV.pdb ==> PROTEIN folder


In [ ]:
#@title **5.2. Extract protein**
#@markdown This generate **clean** protein structure for molecular docking.\
#@markdown The protein chains will be extracted into separate files.

PDB_prot = PDB_ID + "_prot"
PDB_prot_pdb = PDB_prot + ".pdb"
PDB_prot_pdb_Pfile = os.path.join(PRT_FLD, PDB_prot_pdb)

selected = ["ATOM"]
with open(PDB_prot_pdb_Pfile, "w") as o, open(PDB_pdb_Pfile, "r") as i:
    for line in i:
        row = line.split()
        if any(a == row[0] for a in selected):
            o.write(line)

print(f"+ Protein extracted: {PDB_pdb} --> {PDB_prot_pdb}")
seperate_chain(PDB_prot_pdb_Pfile)

+ Protein extracted: 8ONV.pdb --> 8ONV_prot.pdb
+ Chains detected: 1 (A)
+ 8ONV_prot_A.pdb ==> PROTEIN folder


In [ ]:
#@title **5.3. Display 3D protein** { run: "auto" }
#@markdown Enter the **< Protein >** to be viewed.

View = "8ONV_prot_A" #@param {type:"string"}
Model = "cartoon" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "yellow" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "" #@param {type:"string"}
Residues_colour = "green" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_chains = True #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
View_outline = True #@param {type:"boolean"}
pview_Pfile = os.path.join(PRT_FLD, f"{View}.pdb")

VIEW_PROT(inputP=pview_Pfile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showChain=Show_all_chains,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          outline=View_outline)

+ Showing 8ONV_prot_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **5.4. Parameterise protein with Gasteiger charges**
#@markdown Enter the **< Protein >** to be parameterised.\
#@markdown This generate a **`protein.pdbqt`** file after:
#@markdown + Addition of Gasteiger Partial Charge
#@markdown + pH 7.4
#@markdown + Removal of non-polar hydrogens

Target_protein = "8ONV_prot_A" #@param {type:"string"}
PROT_pdb = Target_protein + ".pdb"
PROT_pqr = Target_protein + ".pqr"
PROT_pqr_Pfile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pqr_Dfile = os.path.join(DCK_FLD, PROT_pqr)

PROT_pdbqt = Target_protein + ".pdbqt"
PROT_pdb_Pfile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pdb_Dfile = os.path.join(DCK_FLD, PROT_pdb)
PROT_pdbqt_Dfile = os.path.join(DCK_FLD, PROT_pdbqt)

with Hide():
  while(True):
    try:
      !pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 $PROT_pqr_Pfile $PROT_pqr_Dfile
      !python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py" -r $PROT_pqr_Dfile -o $PROT_pdbqt_Dfile -A hydrogens -U nphs_lps -v
      break
    except:
      print('Can not handle missing residues')
      !python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py" -r $PROT_pqr_Pfile -o $PROT_pdbqt_Dfile -A hydrogens -U nphs_lps -v

  #if os.path.isdir(PROT_pqr_Dfile) == False:

  shutil.copy(PROT_pdb_Pfile, PROT_pdb_Dfile)

print(f"+ Protein parameterised: {PROT_pdb} --> {PROT_pdbqt}")
print(f"+ {PROT_pdb} ==> DOCKING folder")
print(f"+ {PROT_pdbqt} ==> DOCKING folder")

+ Protein parameterised: 8ONV_prot_A.pdb --> 8ONV_prot_A.pdbqt
+ 8ONV_prot_A.pdb ==> DOCKING folder
+ 8ONV_prot_A.pdbqt ==> DOCKING folder


In [ ]:
#@title **5.5. Generate experimental ligand PDB file**: Extracting the experimental ligand for later comparison with the binding modes of docked ligand.
#@markdown Enter the **< Keyword >** assigned to query for the experimental
#@markdown ligand in the **`pdb`** fle.

Keyword = "VU6" #@param {type:"string"}
Elig_pdb = Keyword + ".pdb"
Elig_pdb_Efile = os.path.join(EXP_FLD, Elig_pdb)

selected = [Keyword]
with open(Elig_pdb_Efile, "w") as o, open(PDB_pdb_Pfile, "r") as i:
    for line in i:
        row = line.split()
        if any(a in row for a in selected):
            o.write(line)

print(f"+ Experimental ligand extracted: {PDB_pdb} --> {Elig_pdb}")

+ Experimental ligand extracted: 8ONV.pdb --> VU6.pdb


In [ ]:
#@title **5.6. Extract co-crystal ligands**
#@markdown This generate **clean** corystal ligands for comparison.\
#@markdown The experimental ligands will be extracted into separate files.

seperate_chain(Elig_pdb_Efile)

+ Chains detected: 1 (A)
+ VU6_A.pdb ==> EXPERIMENTAL folder


In [ ]:
#@title **5.7. Display 3D ligand** {run: "auto"}
#@markdown Enter the **< Corystal Ligand >** to be viewed.

View = "VU6_A" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
View_outline = True #@param {type:"boolean"}
eview_Efile = os.path.join(EXP_FLD, f"{View}.pdb")

VIEW_ELIG(inputE=eview_Efile,
          showAtomLabel=Show_atom_labels,
          outline=View_outline)

+ Showing VU6_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **5.8. Choose corystal ligand**
#@markdown Enter the **< Target corystal Ligand >** for later comparison.

Target_experimental_ligand = "VU6_A" #@param {type:"string"}
EXP_pdb = Target_experimental_ligand + ".pdb"
EXP_pdb_Efile = os.path.join(EXP_FLD, EXP_pdb)
EXP_pdb_Dfile = os.path.join(DCK_FLD, EXP_pdb)

with Hide():
    shutil.copy(EXP_pdb_Efile, EXP_pdb_Dfile)

print(f"+ {EXP_pdb} ==> DOCKING folder")

+ VU6_A.pdb ==> DOCKING folder


# **6. Target ligand preparation**

In [ ]:
#@title **6.1. Generate SMILES file**
#@markdown Enter the ligand **< ID >** and **< SMILES >** to be docked. \
#@markdown This generate **`smi`** file for the ligand.

Ligand_ID = "VU6" #@param {type:"string"}
Ligand_SMILES = "C[C@H]1CNCCN1c2nccc(n2)c3c4c(on3)[C@@]5(CCC4)CCCc6c5c(c(s6)N)C#N" #@param {type:"string"}

LIG_ID = Ligand_ID
LIG_smi = LIG_ID + ".smi"
LIG_mol2 = LIG_ID + ".mol2"
LIG_pdbqt = LIG_ID + ".pdbqt"
LIG_smi_Lfile = os.path.join(LIG_FLD, f"{LIG_smi}")
LIG_mol2_Lfile = os.path.join(LIG_FLD, f"{LIG_mol2}")
LIG_pdbqt_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_pdbqt}")

with open(LIG_smi_Lfile, "w") as o:
    o.write(Ligand_SMILES)

print(f"+ {LIG_smi} ==> LIGAND folder")

+ VU6.smi ==> LIGAND folder


In [ ]:
#@title **6.2. Display 3D ligand** {run: "auto"}
#@markdown Enter the **< Ligand >** to be viewed.

View = "VU6" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
Show_hydrogens = True #@param {type:"boolean"}
View_outline = True #@param {type:"boolean"}
lview_Lfile = os.path.join(LIG_FLD, f"{View}.smi")

VIEW_LIG(inputL=lview_Lfile,
         showHs=Show_hydrogens,
         showAtomLabel=Show_atom_labels,
         outline=View_outline)

+ Showing VU6.smi



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **6.3. Parameterize ligand(s) with Gasteiger charges**
#@markdown Select **< Force Field >** for energy minimization. \
#@markdown This generate **`ligand.mol2`** and **`ligand.pdbqt`** file after:
#@markdown + Energy minization at 10,000 iterations
#@markdown + Addition of Gasteiger Partial Charge
#@markdown + Removal of non-polar hydrogens

# NOTE: The prepare_ligand4.py does not recognise the mol2 file location in
#       LIGAND folder for some reason. Hence, the mol2 file was generated in
#       /content/ and then move back into LIGAND folder after parameterization.

Select_force_field = "GAFF" #@param ["GAFF", "Ghemical", "MMFF94", "MMFF94s", "UFF"]

LIG_Dfld = os.path.join(DCK_FLD, f"{Ligand_ID}")
LIG_mol2_DRfile = os.path.join(DIR, f"{LIG_mol2}")
os.makedirs(LIG_Dfld, exist_ok=True)

with Hide():
    !obabel $LIG_smi_Lfile -O $LIG_mol2_DRfile --gen3d --OB_RANDOM_SEED 42 --best --canonical \
    --minimize --ff $Select_force_field --steps 10000 --sd
    !python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py" -l $LIG_mol2_DRfile -o $LIG_pdbqt_Dfile -U nphs_lps -v
    shutil.move(LIG_mol2_DRfile, LIG_mol2_Lfile)
    shutil.copy(LIG_pdbqt_Dfile, DCK_FLD)


print(f"+ {LIG_smi} ==> LIGAND folder")
print(f"+ {LIG_pdbqt} ==> DOCKING folder")



+ VU6.smi ==> LIGAND folder
+ VU6.pdbqt ==> DOCKING folder


In [ ]:
#@title **7. Grid Preparation for Molecular Docking**

## Install dependencies and import libary ##
os.chdir('/content/KRAS-G13D-BI_2493/DOCKING')
!cp '/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_gpf4.py' /content/KRAS-G13D-BI_2493/DOCKING
!wget https://autodock.scripps.edu/wp-content/uploads/sites/56/2021/10/autodocksuite-4.2.6-x86_64Linux2.tar
!tar -xvf /content/KRAS-G13D-BI_2493/DOCKING/autodocksuite-4.2.6-x86_64Linux2.tar
!cp /content/KRAS-G13D-BI_2493/DOCKING/x86_64Linux2/autogrid4 /content/KRAS-G13D-BI_2493/DOCKING

--2023-08-01 03:23:03--  https://autodock.scripps.edu/wp-content/uploads/sites/56/2021/10/autodocksuite-4.2.6-x86_64Linux2.tar
Resolving autodock.scripps.edu (autodock.scripps.edu)... 192.26.252.19
Connecting to autodock.scripps.edu (autodock.scripps.edu)|192.26.252.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 743424 (726K) [application/x-tar]
Saving to: ‘autodocksuite-4.2.6-x86_64Linux2.tar.1’

autodocksuite-4.2.6 100%[===================>] 726.00K  4.59MB/s    in 0.2s    

2023-08-01 03:23:04 (4.59 MB/s) - ‘autodocksuite-4.2.6-x86_64Linux2.tar.1’ saved [743424/743424]

x86_64Linux2/autodock4
x86_64Linux2/autogrid4


In [ ]:
#@title **7.1 Run Grid calculation**

# -- Start install --
start = timeit.default_timer()

## Execute Grid calculation for molecular docking ##
!python2 /content/KRAS-G13D-BI_2493/DOCKING/prepare_gpf4.py -l /content/KRAS-G13D-BI_2493/DOCKING/VU6.pdbqt -r /content/KRAS-G13D-BI_2493/DOCKING/8ONV_prot_A.pdbqt -p spacing='0.375' -p npts='126,126,126' -p gridcenter=auto -o grid.gpf
!./autogrid4 -p grid.gpf -l grid.glg

stop = timeit.default_timer()
# -- End install --

time_elp = stop - start

print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")

+ Operation completed
+ Time elapsed: 00m 52s


In [ ]:
#@title **8. Molecular Docking Execution by AutoDock-GPU**
## Git Clone AutoDock-GPU ##
!git clone https://github.com/ccsb-scripps/AutoDock-GPU.git

Cloning into 'AutoDock-GPU'...
remote: Enumerating objects: 6431, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 6431 (delta 68), reused 94 (delta 51), pack-reused 6297
Receiving objects: 100% (6431/6431), 45.43 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (4258/4258), done.


In [ ]:
#@title **8.1 Complie AutoDock-GPU**

## Complie AutoDock-GPU  ##

%%bash
cd AutoDock-GPU/
export GPU_INCLUDE_PATH=/usr/local/cuda/include
export GPU_LIBRARY_PATH=/usr/local/cuda/lib64
make DEVICE=CUDA NUMWI=128


rm -f ./host/inc/performdocking.h ./host/src/performdocking.cpp
Building adgpu_analysis ...
g++ \
./host/src/calcenergy.cpp ./host/src/getparameters.cpp ./host/src/main.cpp ./host/src/miscellaneous.cpp ./host/src/processgrid.cpp ./host/src/processligand.cpp ./host/src/processresult.cpp ./host/src/setup.cpp \
-std=c++11 -I./common -I./host/inc -DVERSION=\"v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47\" \
-o./bin/adgpu_analysis \
 -O3 -DTOOLMODE 
DEVICE is set to GPU
 
CPU_INCLUDE_PATH is undefined
CPU_LIBRARY_PATH is undefined
 
GPU_INCLUDE_PATH is set to /usr/local/cuda/include
GPU_LIBRARY_PATH is set to /usr/local/cuda/lib64
 
nvcc -DN128WI  -use_fast_math --ptxas-options="-v" -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -std=c++11 -I./common -I./host/inc -I/usr/local/cuda/include -I./cuda  -c ./cuda/kernels.cu
ln -sf performdocking.h.Cuda ./host/inc/performdocking.h
ln -sf perform

In file included from /usr/include/string.h:535,
                 from ./host/inc/getparameters.h:30,
                 from ./host/src/getparameters.cpp:32:
In function ‘char* strncat(char*, const char*, size_t)’,
    inlined from ‘int parse_dpf(Dockpars*, Gridinfo*, FileList&, bool)’ at ./host/src/getparameters.cpp:213:16:
/usr/include/x86_64-linux-gnu/bits/string_fortified.h:138:34: warning: ‘char* __builtin_strncat(char*, const char*, long unsigned int)’ specified bound depends on the length of the source argument [-Wstringop-overflow=]
  138 |   return __builtin___strncat_chk (__dest, __src, __len,
      |          ~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~
  139 |                                   __glibc_objsize (__dest));
      |                                   ~~~~~~~~~~~~~~~~~~~~~~~~~
./host/src/getparameters.cpp: In function ‘int parse_dpf(Dockpars*, Gridinfo*, FileList&, bool)’:
./host/src/getparameters.cpp:211:77: note: length computed here
  211 |                    

In [ ]:
#@title **8.2 Docking Target protein and ligand using AutoDock-GPU**

## Copy executable program to working directory   ##
!cp /content/KRAS-G13D-BI_2493/DOCKING/AutoDock-GPU/bin/autodock_gpu_128wi /content/KRAS-G13D-BI_2493//DOCKING

## Run Docking using AutoDock-GPU ##
# -- Start running --
start = timeit.default_timer()

!./autodock_gpu_128wi --ffile 8ONV_prot_A.maps.fld --lfile VU6.pdbqt --nrun 50 --psize 200 --resnam dock1st --gbest --contact_analysis

stop = timeit.default_timer()
# -- End install --

time_elp = stop - start

print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")

AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              Tesla T4
Available memory on device:               14996 MB (total: 15101 MB)

CUDA Setup time 0.329904s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 480000
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.094260s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 480000 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          200 |   -3.76 kcal/mol |   -4.19 +/-    0.15 kcal/mol |       7 |   -4.45 kcal/mol
          5 |       134875 |   -3.76 kcal/mol |   -6.00 +/-    1.38 

#**8.3 Post-Docking Analysis: convert xml2csv**
Link: https://www.convertcsv.com/xml-to-csv.htm

In [ ]:
## Load AutoDock-GPU score into DataFrame via pandas and plotly
!pip install plotly
import plotly.express as px
import pandas as pdf
df = pd.read_csv('/content/KRAS-G13D-BI_2493/DOCKING/dock1st.csv')
df

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached plotly-5.15.0-py2.py3-none-any.whl (15.5 MB)


,free_NRG_binding,final_intermol_NRG,internal_ligand_NRG,torsonial_free_NRG,tran0,axisangle0,ndihe,dihe0,_id
0,-13.05,-14.24,-0.73,1.19,27.707537 15.032344 15.394994,0.42891333 -0.90324369 0.01357158 216.911255,4,193.737671 205.200623 308.910706 348.002563,18
1,-12.99,-14.19,-0.75,1.19,27.607452 15.036677 15.384476,-0.41802468 0.90790786 -0.03096269 143.851669,4,193.452148 208.395767 260.854614 347.474426,33
2,-12.84,-14.03,-0.77,1.19,27.346802 15.184072 15.321194,-0.37179567 0.92829288 0.00634893 143.742111,4,183.356842 211.571320 287.635559 330.644257,34
3,-12.71,-13.91,-0.78,1.19,27.317099 15.192420 15.323556,-0.37497726 0.92701538 -0.00587676 144.208252,4,184.672729 212.545715 279.730225 87.557320,27
4,-12.67,-13.86,-0.79,1.19,27.335512 15.310066 15.384748,0.36710786 -0.92987272 0.02384408 214.094482,4,183.819336 215.744263 256.076660 75.537086,6
5,-12.50,-13.69,-0.70,1.19,27.108602 15.152464 15.206956,0.36597923 -0.93054269 0.01222720 213.576309,4,297.525482 201.893646 281.389954 93.604729,1
6,-12.17,-13.36,-0.63,1.19,27.628824 15.021501 15.211620,-0.42239036 0.90638429 -0.00734107 145.202850,4,227.867096 184.109955 301.660187 355.521301,40
7,-12.15,-13.34,-0.81,1.19,26.998203 15.138140 15.426311,0.32372399 -0.94613076 0.00627343 217.704819,4,154.792892 221.445831 310.346130 90.656059,4
8,-12.04,-13.24,-0.75,1.19,27.116035 15.061585 15.209270,-0.32030723 0.94262961 0.09408872 134.894531,4,160.734360 208.362534 298.847382 98.148674,41
9,-11.84,-13.03,-0.68,1.19,27.209304 15.051815 15.094741,-0.32857734 0.93039996 0.16245875 135.839828,4,217.855698 165.470062 331.878815 49.892017,5


In [ ]:
# Assuming you have loaded your data into the DataFrame "df"

# Create the histogram using Plotly Express
fig = px.histogram(df, x='free_NRG_binding', nbins=20, title='Free Energy Binding Histogram')

# Show the plot
fig.show()


In [ ]:
## Dock score Histrogram visualization ##
# Create the box plot using Plotly Express
fig = px.box(df, y='free_NRG_binding', title='Free Energy Binding Box Plot')

# Show the plot
fig.show()

#**9.Save Docking Result Folder to Google Drive**

In [ ]:
#@title **9.1. Import Google Drive**

# Flush and mount GDrive
with Hide():
  drive.flush_and_unmount()
  drive.mount("/content/drive", force_remount=True)

print(f"> Mounted at /content/drive")

> Mounted at /content/drive


In [ ]:

#@title **Store result in Google Drive**
#@markdown Enter the **< Destination Path >** for saving. \
#@markdown This save all docking data into Google Drive.

Destination = "/content/drive/MyDrive/KRAS-G13D-BI_2493" # @param {type:"string"}
DST_FLD = os.path.join(Destination, Job_name)
shutil.copytree(WRK_DIR, DST_FLD)

print(f"> Data saved at {DST_FLD}")

> Data saved at /content/drive/MyDrive/KRAS-G13D-BI_2493/KRAS-G13D-BI_2493
